In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env2 import ConceptData
from create_data import addFile
import agents
import sender

In [2]:
sender_encoder = sender.Sender(num_options=10, batch_size=32)
sender_only_target = sender.SenderOnlyTarget(batch_size=32)
sender = sender.Sender_LSTM(vocab_size=99, embed_dim=44, num_cells=1, hidden_size=1, max_len=10, see_all_input=True)
receiver_decoder = agents.Receiver(num_options=10)
receiver_encoder = agents.Receiver_LSTM(agent=receiver_decoder, vocab_size=99, embed_dim=44, hidden_size=44, masking_value=99)

In [3]:
vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)
    
data = ConceptData(vocabulary, 9, 32)

In [4]:
input_concepts, sender_input, targets, receiver_input = data.getInput()
#print(sender_input.shape)

shape sender input:
(32, 10, 1, 595)


In [5]:
sender_encoded = sender_encoder(sender_input)
message, logits, entropy, state_h = sender(sender_encoded)
print("message:")
print(message)
print("logits:")
print(logits)
print("entropy:")
print(entropy)
print("state_h:")
print(state_h)

32
input-shape:
(32, 10, 44)
message:
tf.Tensor(
[[86 22 95 26 89 30 90  8 96  0  0  0  0  0  0  0  0  0]
 [73 93 89 52 36 97 57 58 72  0  0  0  0  0  0  0  0  0]
 [82 87  9 37 90 62 62 24 37  0  0  0  0  0  0  0  0  0]
 [80 46 82 48 16 71 79  1 18  0  0  0  0  0  0  0  0  0]
 [ 0 93 28 98 50 15 12 80 30  0  0  0  0  0  0  0  0  0]
 [75 92 28  3 17 34 97  3 81  0  0  0  0  0  0  0  0  0]
 [83 69 78 15 72 97 78 66 65  0  0  0  0  0  0  0  0  0]
 [82 80 54 64 47 12 82 25 35  0  0  0  0  0  0  0  0  0]
 [11 72 57 49 98 82  1 46 86  0  0  0  0  0  0  0  0  0]
 [47 53 90 75 37 74 28 45 89  0  0  0  0  0  0  0  0  0]
 [14 51 52 65 29 92 63 32 36  0  0  0  0  0  0  0  0  0]
 [63 61 44 55 76 56  6 55 72  0  0  0  0  0  0  0  0  0]
 [72 79 68 31 51 15 65 54 60  0  0  0  0  0  0  0  0  0]
 [80 62 21 62 88 77 27  5 74  0  0  0  0  0  0  0  0  0]
 [41 70 31 92 89 83  9 20 39  0  0  0  0  0  0  0  0  0]
 [43 30  0 53 48 87 97 36  9  0  0  0  0  0  0  0  0  0]
 [41 77 95 91 93 51 34 74 42  0  0  0  

In [11]:

#einer- Dimension rausbekommen
receiver_input = tf.squeeze(tf.convert_to_tensor(receiver_input))
#richtige Reihenfolge der Dimensionen
receiver_input2 = tf.transpose(receiver_input, [1,0,2])
receiver_input2 = receiver_input2.numpy()

sample, log_prob_r, entropy_r, last_hidden = receiver_encoder(message=message, batch_size=32, max_len=10, receiver_input=receiver_input2)



# receiver_output = []
# for i in range(32):
#     receiver_output.append(input_concepts[i][sample[i]])
    
# print(receiver_output)

10
tf.Tensor(
[[[0.46503904 0.48109826 0.4632852  ... 0.5083121  0.51310146 0.54281294]
  [0.49911818 0.5032203  0.46365374 ... 0.539481   0.5508973  0.5004307 ]
  [0.49048337 0.5083795  0.49104506 ... 0.5332859  0.5219097  0.5133203 ]
  ...
  [0.43640858 0.4857279  0.51841664 ... 0.47890225 0.4592931  0.5207629 ]
  [0.45821285 0.571895   0.46881288 ... 0.5938291  0.5794575  0.5205762 ]
  [0.49823526 0.45213583 0.4778923  ... 0.45134085 0.49092093 0.5393748 ]]

 [[0.54697686 0.51316184 0.48660976 ... 0.5330489  0.52417254 0.5466072 ]
  [0.54025954 0.48826024 0.5462842  ... 0.5180493  0.5363151  0.52734226]
  [0.4509802  0.37081558 0.50329745 ... 0.5212341  0.42178887 0.59588474]
  ...
  [0.50463545 0.44076648 0.5236413  ... 0.48481438 0.50882554 0.52959734]
  [0.49771783 0.47021088 0.45053947 ... 0.5687058  0.472825   0.4506606 ]
  [0.5415366  0.45454457 0.49764884 ... 0.48360354 0.49504277 0.4991563 ]]

 [[0.5425021  0.5839589  0.4896182  ... 0.4795553  0.49829665 0.5197234 ]
  [0.508

tf.Tensor([1 0 8 8 3 1 7 2 0 7 0 1 8 7 6 4 9 7 5 3 0 9 1 1 6 7 2 7 2 2 4 0], shape=(32,), dtype=int32)
